Vamos a utilizar el mismo procedimiento usado con los _tweets_ en otros textos

In [1]:
import string
import re
from itertools import product

import pandas as pd
 
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

**Limpieza**

In [2]:
def camel_case(example):      
    if  any(x in example for x  in string.punctuation)==True:
        return False
    else:
        if any(list(map(str.isupper, example[1:-1]))) and not all(list(map(str.isupper, example[1:-1]))):
            return True
        else:
            return False

def camel_case_split(word):
    idx = list(map(str.isupper, word))
    case_change = [0]
    for (i, (x, y)) in enumerate(zip(idx, idx[1:])):
        if x and not y:  
            case_change.append(i)
        elif not x and y:  
            case_change.append(i+1)
    case_change.append(len(word))
    return [word[x:y] for x, y in zip(case_change, case_change[1:]) if x < y]

In [3]:
nlp = spacy.load("es_core_news_lg")
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub('http\S*','',tweet)
    
    # remove users
    tweet = re.sub(r'@[a-zA-Z0-9]+', '', tweet)
    
    # split camel case
    camel_case_words = [word[1:] for word in re.findall(r'#[a-zA-Z]+', tweet) if camel_case(word[1:])]
    splited_words = [' '.join(camel_case_split(word[1:])) for word in re.findall(r'#[a-zA-Z]+', tweet) if camel_case(word[1:])]
    for camel, splited in zip(camel_case_words, splited_words):
        tweet = tweet.replace(camel, splited)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # remove single letters
    tweet = re.sub(r"\b[a-zA-Z]\b", "", tweet)
 
    # remove multiple spaces
    tweet = re.sub(r"\s+", " ", tweet)

    # tokenize tweets
    tweet_tokens = nlp(tweet)
 
    tweets_clean = [word.lemma_ for word in tweet_tokens if not word.is_stop and word.pos_ != 'PUNCT']
 
    return ' '.join(tweets_clean).lower().strip()

**Clasificación**

In [4]:
palabras = pd.read_feather('../data/diccionario_palabras_ods')

words_list = palabras[['ODS.1', 'PALABRAS']].dropna().groupby('ODS.1')['PALABRAS'].apply(list)
words_list_nlp = [nlp(' '.join(words).lower()) for words in words_list]

## [Objetivo 1: Fin de la pobreza](https://www.undp.org/content/undp/es/home/sustainable-development-goals/goal-1-no-poverty.html)

In [5]:
ods_1 = """
Erradicar la pobreza en todas sus formas sigue siendo uno de los principales desafíos que enfrenta la humanidad. Si bien la cantidad de personas que viven en la extrema pobreza disminuyó en más de la mitad entre 1990 y 2015, aún demasiadas luchan por satisfacer las necesidades más básicas.

A nivel mundial, alrededor de 736 millones de personas aún viven con menos de US$1,90 al día (2015) y muchos carecen de acceso a alimentos, agua potable y saneamiento adecuados. El crecimiento económico acelerado de países como China e India ha sacado a millones de personas de la pobreza, pero el progreso ha sido disparejo. La posibilidad de que las mujeres vivan en situación de pobreza es desproporcionadamente alta en relación con los hombres, debido al acceso desigual al trabajo remunerado, la educación y la propiedad.

Los avances también han sido limitados en otras regiones, como Asia Meridional y África subsahariana, donde vive el 80% de la población mundial que se encuentra en condiciones de extrema pobreza. Además, nuevas amenazas que plantean el cambio climático, los conflictos y la inseguridad alimentaria necesitan mayores esfuerzos para sacar a las personas de la pobreza.

Los Objetivos de Desarrollo Sostenible constituyen un compromiso audaz para finalizar lo que comenzamos y terminar con la pobreza en todas sus formas y dimensiones para 2030. Esto requiere enfocarse en los más vulnerables, aumentar el acceso a los recursos y servicios básicos y apoyar a las comunidades afectadas por conflictos y desastres relacionados con el clima.
"""

In [6]:
ods_1 = clean_tweets(ods_1)
ods_1

'erradicar pobreza formar principal desafío enfrentar humanidad cantidad personar vivir extremo pobreza disminuir mitad 1990 2015 demasiar luchar satisfacer necesidad básico nivel mundial 736 millón personar vivir us,90 2015 carecer acceso alimento aguar potable saneamiento adecuar crecimiento económico acelerar país china india sacar millón personar pobreza progresar disparejo posibilidad mujer vivir situación pobreza desproporcionadamente alto relación hombre acceso desigual remunerar educación propiedad avanzar limitar región asia meridional áfrica subsahariano vivir 80% población mundial condicionar extremo pobreza amenazar plantear cambiar climático conflicto inseguridad alimentario necesitar mayor esfuerzo sacar personar pobreza objetivos desarrollo sostenible constituir compromiso audaz finalizar comenzar terminar pobreza formar dimensionar 2030 requerir enfocarse vulnerable aumentar acceso recurso servicio básico apoyar comunidad afectar conflicto desastre relacionar clima'

In [7]:
ods_1_sim = [nlp(ods_1).similarity(words) for words in words_list_nlp]
ods_1_sim

[0.779285550590821,
 0.783839994867686,
 0.8118746492257268,
 0.7394701469184014,
 0.7851740340590067,
 0.5802161881880262,
 0.6060942946677514,
 0.8403538764354862,
 0.7201978175467083,
 0.8364762405365745,
 0.6893683361546529,
 0.7483268347304876,
 0.6654156393107648,
 0.5828273059427406,
 0.7041392289893651,
 0.7797578445934781,
 0.5716903765988367]

## [Objetivo 2: Hambre cero](https://www.undp.org/content/undp/es/home/sustainable-development-goals/goal-2-zero-hunger.html)

In [8]:
ods_2 = """
Debido al rápido crecimiento económico y al aumento de la productividad agrícola en las últimas dos décadas, el número de personas desnutridas disminuyó casi a la mitad. Muchos países en desarrollo que sufrían hambrunas están ahora en condiciones de satisfacer las necesidades nutricionales de los más vulnerables. Regiones como Asia Central y Oriental y América Latina y el Caribe han avanzado enormemente en la erradicación del hambre.

Desgraciadamente, el hambre y la desnutrición siguen siendo grandes obstáculos para el desarrollo de muchos países. Se estima que 821 millones de personas sufrían de desnutrición crónica al 2017, a menudo como consecuencia directa de la degradación ambiental, la sequía y la pérdida de biodiversidad. Más de 90 millones de niños menores de cinco años tienen un peso peligrosamente bajo. La desnutrición y la inseguridad alimentaria parece estar incrementándose tanto en casi todas las de regiones de África, como en América del Sur.

Los Objetivos de Desarrollo Sostenible buscan terminar con todas las formas de hambre y desnutrición para 2030 y velar por el acceso de todas las personas, en especial los niños, a una alimentación suficiente y nutritiva durante todo el año. Esta tarea implica promover prácticas agrícolas sostenibles a través del apoyo a los pequeños agricultores y el acceso igualitario a la tierra, la tecnología y los mercados. Además, se requiere el fomento de la cooperación internacional para asegurar la inversión en la infraestructura y la tecnología necesaria para mejorar la productividad agrícola.
"""

In [9]:
ods_2 = clean_tweets(ods_2)
ods_2_sim = [nlp(ods_2).similarity(words) for words in words_list_nlp]
ods_2_sim

[0.7986402178618875,
 0.8261174618843236,
 0.8339785892698295,
 0.7365200106428437,
 0.7737188275086234,
 0.6212504577973911,
 0.6369638681045555,
 0.8523199401349276,
 0.7633219579811927,
 0.8272678893712321,
 0.7442665558127006,
 0.7879045688323357,
 0.6993982787070482,
 0.6270059933401393,
 0.7551870985029612,
 0.764097943001074,
 0.6112705594506679]

## [El Govern pretende cerrar Punta Ballena y la calle de la cerveza](https://www.diariodemallorca.es/mallorca/2020/07/14/govern-pretende-cerrar-punta-ballena/1523350.html)

In [10]:
noticia = """
El Govern balear pretende ordenar el cierre de las dos calles de Mallorca más conflictivas a causa de las aglomeraciones de turísticas incumpliendo las medidas sanitarias que se están produciendo y cuyas imágenes están dando la vuelta al mundo: Punta Ballena en Magaluf y la conocida como calle de la cerveza en la Platja de Palma. El objetivo es lanzar un mensaje de tranquilidad a los países emisores, especialmente a Alemania, respecto a la imagen de las islas como destino seguro, muy deteriorada durante los últimos días a causa de los vídeos que han circulado con fiestas ilegales y comportamientos incívicos.

Los consellers de Turismo y de Administraciones Públicas, Iago Negueruela e Isabel Castro respectivamente, han comparecido hoy para anunciar un "endurecimiento" de las iniciativas destinadas a evitar las vulneraciones de las medidas sanitarias que se han aprobado, y han concretado que éstas se van a centrar en las zonas tipificadas como mas afectadas por el denominado turismo de excesos, como son Platja de Palma, Magaluf y Sant Antoni de Eivissa. Sin embargo, se han negado a avanzar qué alcance va a tener ese "endurecimiento", al señalar que se está terminando de definir y que previsiblemente se hará público durante el día de mañana. Fuentes del sector turístico han reconocido que entre las iniciativas que se está planteando el Govern aparece el cierre de las dos calles antes señaladas.

Los dos representantes del Ejecutivo balear no han ocultado el daño que el comportamiento irresponsable de unos pocos puede hacer tanto a los residentes en las islas como a sus visitantes, de ahí el objetivo de atajarlo.

Este anuncio por parte de Negueruela y Castro se ha producido tras una reunión celebrada hoy con representantes de las patronales hoteleras y de los sindicatos, que ha sido "muy tensa" en algunos momentos por el rechazo de la parte empresarial a algunas de las iniciativas que ha tomado el Gabinete de Francina Armengol, como es la obligatoriedad de llevar siempre la mascarilla en espacios públicos salvo algunas excepciones.
"""

In [11]:
noticia = clean_tweets(noticia)
noticia_sim = [nlp(noticia).similarity(words) for words in words_list_nlp]
noticia_sim

[0.6433895807139273,
 0.6479185710223566,
 0.707448111692085,
 0.6593374101530624,
 0.6757161633462677,
 0.5312402277991756,
 0.5129986966287136,
 0.7262882707644215,
 0.6436464179578479,
 0.7026063097073116,
 0.6367470984867267,
 0.6718119150224988,
 0.6137344204541554,
 0.5538252924340162,
 0.6264353700737133,
 0.6987110142551988,
 0.5187347770023865]

## [El Mallorca lo tiene un poco más complicado](https://www.diariodemallorca.es/rcd-mallorca/2020/07/14/complicado/1523250.html)

In [12]:
deportes = """
Seis puntos. Esa es la cifra que debe conseguir el Mallorca para apurar sus ya escasas opciones de permanencia. No queda otra. El Alavés, tras su empate ante el Getafe (0-0), marca el corte de la salvación con 36 puntos, igualado con el Celta, cuatro más de los que el equipo de Vicente Moreno tiene en su haber. Complicado que el Mallorca se salve, cierto, pero no es imposible.

La situación clasificatoria ha pasado de color gris a negro tras un fin de semana catastrófico que no solo ha eliminado al Eibar de la terna de candidatos al descenso, sino que incluso ha incluido en la fiesta a un conjunto que parecía desahuciado como el Leganés, empatado a puntos con los bermellones y con el golaverage directo a favor. Pese a ello, no queda tiempo para lamentaciones. Todo pasa por conseguir los seis puntos que restan y esperar un gran favor del resto del rivales. Podrían bastar cuatro, las matemáticas así lo dictan, pero las probabilidades necesarias para que se dé el caso son ínfimas.

Lo que es seguro es que el Alavés, que al igual que el conjunto pepinero tiene el golaverage a favor con los bermellones, no puede ganar ningún partido de los que restan y sumar como máximo un punto, mientras que el Leganés no debe vencer en alguno de los encuentros que le quedan por delante para que el Mallorca, si cumple con su trabajo, sea un año más equipo de Primera División. El cuarto equipo en la brega, el Celta, que cuenta con 36 puntos, es el que tiene a priori un mejor calendario. Ya sea venciendo en casa al Levante, que no se juega nada más allá de objetivos internos, o consiguiendo el triunfo ante el Espanyol, ya descendido, en Cornellà, los de Óscar García tienen a tiro sellar su permanencia en Primera.

Como pronto, el Mallorca tendrá que tumbar al Granada este jueves en el Visit Mallorca Estadi, que no será nada fácil. Y un dato importante: si Celta y Alavés consiguen el mismo resultado que se produzca en Son Moix, sea victoria, empate o derrota, el Mallorca será nuevo equipo de Segunda División.

La misión que tiene el Mallorca adopta ya unas dimensiones titánicas. Demasiadas variables en juego y la dependencia de otros resultados convierten lograr la permanencia en una gesta épica. Todo puede pasar en el fútbol. En apenas dos días se decidirá todo.
"""

In [13]:
deportes = clean_tweets(deportes)
deportes_sim = [nlp(deportes).similarity(words) for words in words_list_nlp]
deportes_sim

[0.5014104454057435,
 0.5651478353461546,
 0.5693213403681013,
 0.5366688755709058,
 0.5503250379320013,
 0.38963256698520377,
 0.4439108466611069,
 0.5793873929786267,
 0.4787202090149401,
 0.5421675209289147,
 0.4719464155259662,
 0.5290172132787552,
 0.5205932443057127,
 0.43018135484564807,
 0.48265602497236604,
 0.5194781874594344,
 0.41808651240408895]

## [LEYENDAS DEL ROCK 2021: entradas ya a la venta](https://www.rafabasa.com/2020/07/13/leyendas-del-rock-2021-entradas-ya-a-la-venta/)

In [14]:
concierto = """
¡Las entradas para LEYENDAS DEL ROCK 2021 ya están a la venta! Abono 3 días, camping sombra y piscina. Y lo hacemos con una oferta muy especial, un regalo de 10€ en tokens por abono 3 días adquirido (solo en la red de Ticketmaster). Oferta válida hasta el 31 de julio. También puedes adquirir tu abono a todo color en los puntos habituales.

Puedes adquirir aquí tu abono

Después de las reconfirmaciones de STRATOVARIUS, SARATOGA y JINJER, en las próximas semanas iremos desvelando los nombres que ya teníamos confirmados en 2020 y siguen en cartel, así como el anuncio de nuevas bandas de cara a nuestra edición de 2021. Permanece atento y sobre todo, ¡¡permanece muy heavy!!
"""

In [15]:
concierto = clean_tweets(concierto)
concierto_sim = [nlp(concierto).similarity(words) for words in words_list_nlp]
concierto_sim

[0.2715914235078668,
 0.39471491881475373,
 0.3246180140628708,
 0.39444278506035546,
 0.28493539896253284,
 0.3056374786514312,
 0.3551473116446221,
 0.3780156746859772,
 0.3761442965355766,
 0.29758496859115285,
 0.3742374562828454,
 0.4455249089094482,
 0.44562735335873216,
 0.23151233403490784,
 0.26529118729360046,
 0.21951367590473841,
 0.218246410865622]

Añadimos el texto del resto de ODS.

In [16]:
ods_3 = """
Hemos logrado grandes avances en la lucha contra varias de las principales causas de muerte y enfermedad. La esperanza de vida ha aumentado drásticamente, las tasas de mortalidad infantil y materna han disminuido, hemos cambiado el curso del VIH y la mortalidad debida a la malaria se ha reducido a la mitad.


La buena salud es esencial para el desarrollo sostenible, y la Agenda 2030 refleja la complejidad y la interconexión de ambos. Toma en cuenta la ampliación de las desigualdades económicas y sociales, la rápida urbanización, las amenazas para el clima y el medio ambiente, la lucha continua contra el VIH y otras enfermedades infecciosas, y los nuevos problemas de salud, como las enfermedades no transmisibles. La cobertura universal de salud será integral para lograr el ODS 3, terminar con la pobreza y reducir las desigualdades. Las prioridades de salud global emergentes que no se incluyen explícitamente en los ODS, incluida la resistencia a los antimicrobianos, también demandan acción.

Sin embargo, el mundo no está bien encaminado para alcanzar los ODS relacionados con la salud. El progreso ha sido desigual, tanto entre países como dentro de ellos. Sigue habiendo una discrepancia de 31 años entre los países con la esperanza de vida más corta y la más larga. Si bien algunos han logrado avances impresionantes, los promedios nacionales ocultan el hecho de que algunas poblaciones, grupos y comunidades se están quedando atrás. Los enfoques multisectoriales, basados ​​en los derechos y con perspectiva de género, son esenciales para abordar las desigualdades y asegurar una buena salud para todas las personas.
"""

In [17]:
ods_4 = """
Desde 2000 se ha registrado un enorme progreso en la meta relativa a la educación primaria universal. La tasa total de matrícula alcanzó el 91% en las regiones en desarrollo en 2015 y la cantidad de niños que no asisten a la escuela disminuyó casi a la mitad a nivel mundial. También ha habido aumentos significativos en las tasas de alfabetización y más niñas que nunca antes asisten hoy a la escuela. Sin duda, se trata de logros notables.

Sin embargo, el progreso también ha sido difícil en las regiones en desarrollo debido a los altos niveles de pobreza, conflictos armados y otras emergencias. En Asia Occidental y el Norte de África, los conflictos armados en curso han aumentado la proporción de niños que no asisten a la escuela, constituyendo una tendencia preocupante.

Si bien África subsahariana consiguió los avances más notables en la matriculación en la escuela primaria entre todas las regiones en desarrollo (de 52% en 1990 a 78% en 2012), aún hay grandes disparidades, especialmente entre las zonas rurales y urbanas. Por su parte, los niños de los hogares más pobres tienen hasta cuatro veces más probabilidades de no asistir a la escuela que aquellos provenientes de familias con más recursos.

El objetivo de lograr una educación inclusiva y de calidad para todos se basa en la firme convicción de que la educación es uno de los motores más poderosos y probados para garantizar el desarrollo sostenible. Con este fin, el objetivo busca asegurar que todas las niñas y niños completen su educación primaria y secundaria gratuita para 2030. También aspira a proporcionar acceso igualitario a formación técnica asequible y eliminar las disparidades de género e ingresos, además de lograr el acceso universal a educación superior de calidad.
"""

In [18]:
ods_5 = """
Poner fin a todas las formas de discriminación contra las mujeres y niñas no es solo un derecho humano básico, sino que además es crucial para el desarrollo sostenible. Se ha demostrado una y otra vez que empoderar a las mujeres y niñas tiene un efecto multiplicador y ayuda a promover el crecimiento económico y el desarrollo a nivel mundial.

El PNUD le ha otorgado a la igualdad de género un lugar central en su trabajo y hemos visto un progreso notable en los últimos 20 años. Más niñas van a la escuela que hace 15 años, y la mayoría de las regiones ha alcanzado la paridad de género en la educación primaria.

Pero aunque hay más mujeres que nunca en el mercado laboral, todavía hay grandes desigualdades en algunas regiones, y sistemáticamente a las mujeres se les niegan los mismos derechos laborales que tienen los hombres. La violencia y la explotación sexual, la división desigual del trabajo no remunerado -tanto doméstico como en el cuidado de otras personas- y la discriminación en la toma de decisiones en el ámbito público son grandes obstáculos que aún persisten. El cambio climático y los desastres continúan teniendo un efecto desproporcionado en las mujeres y los niños, al igual que el conflicto y la migración.

Garantizar el acceso universal a salud reproductiva y sexual y otorgar a la mujer derechos igualitarios en el acceso a recursos económicos, como tierras y propiedades, son metas fundamentales para conseguir este objetivo. Hoy más mujeres que nunca ocupan cargos públicos, pero alentar a más mujeres para que se conviertan en líderes ayudará a alcanzar una mayor igualdad de género.
"""

In [19]:
ods_6 = """
La escasez de agua afecta a más del 40 por ciento de la población mundial, una cifra alarmante que probablemente crecerá con el aumento de las temperaturas globales producto del cambio climático. Aunque 2.100 millones de personas han conseguido acceso a mejores condiciones de agua y saneamiento desde 1990, la decreciente disponibilidad de agua potable de calidad es un problema importante que aqueja a todos los continentes.

Cada vez más países están experimentando estrés hídrico, y el aumento de las sequías y la desertificación ya está empeorando estas tendencias. Se estima que al menos una de cada cuatro personas se verá afectada por escasez recurrente de agua para 2050.

Con el fin de garantizar el acceso universal al agua potable segura y asequible para todos en 2030, es necesario realizar inversiones adecuadas en infraestructura, proporcionar instalaciones sanitarias y fomentar prácticas de higiene.

Asegurar el agua potable segura y asequible universal implica llegar a más de 800 millones de personas que carecen de servicios básicos y mejorar la accesibilidad y seguridad de los servicios por más de dos mil millones.

En 2015, 4.500 millones de personas carecían de servicios de saneamiento administrados de manera segura (con excrementos adecuadamente dispuestos o tratados) y 2.300 millones carecían incluso de saneamiento básico.
"""

In [20]:
ods_7 = """
Entre 2000 y 2016, la cantidad de personas con acceso a energía eléctrica aumentó de 78 a 87 por ciento, y el número de personas sin enegía bajó a poco menos de mil millones.

Sin embargo, a la par con el crecimiento de la población mundial, también lo hará la demanda de energía accesible, y una economía global dependiente de los combustibles fósiles está generando cambios drásticos en nuestro clima.

Para alcanzar el ODS7 para 2030, es necesario invertir en fuentes de energía limpia, como la solar, eólica y termal y mejorar la productividad energética.  

Expandir la infraestructura y mejorar la tecnología para contar con energía limpia en todos los países en desarrollo, es un objetivo crucial que puede estimular el crecimiento y a la vez ayudar al medio ambiente.
"""

In [21]:
ods_8 = """
Durante los últimos 25 años, la cantidad de trabajadores que viven en condiciones de pobreza extrema ha disminuido drásticamente, pese al impacto de la crisis económica de 2008 y las recesiones globales. En los países en desarrollo, la clase media representa hoy más del 34% del empleo total, una cifra que casi se triplicó entre 1991 y 2015.

Sin embargo, mientras la economía mundial continúa recuperándose presenciamos un crecimiento más lento, un aumento de las desigualdades y un déficit de empleos para absorber la creciente fuerza laboral. Según la Organización Internacional del Trabajo (OIT), en 2015 hay más de 204 millones de personas desempleadas.

Los Objetivos de Desarrollo Sostenible apuntan a estimular el crecimiento económico sostenible mediante el aumento de los niveles de productividad y la innovación tecnológica. Fomentar políticas que estimulen el espíritu empresarial y la creación de empleo es crucial para este fin, así como también las medidas eficaces para erradicar el trabajo forzoso, la esclavitud y el tráfico humano. Con estas metas en consideración, el objetivo es lograr empleo pleno y productivo y un trabajo decente para todos los hombres y mujeres para 2030.
"""

In [22]:
ods_9 = """
La inversión en infraestructura y la innovación son motores fundamentales del crecimiento y el desarrollo económico. Con más de la mitad de la población mundial viviendo en ciudades, el transporte masivo y la energía renovable son cada vez más importantes, así como también el crecimiento de nuevas industrias y de las tecnologías de la información y las comunicaciones.

Los avances tecnológicos también con esenciales para encontrar soluciones permanentes a los desafíos económicos y ambientales, al igual que la oferta de nuevos empleos y la promoción de la eficiencia energética. Otras formas importantes para facilitar el desarrollo sostenible son la promoción de industrias sostenibles y la inversión en investigación e innovación científicas.

Más de 4.000 millones de personas aún no tienen acceso a Internet y el 90 por ciento proviene del mundo en desarrollo. Reducir esta brecha digital es crucial para garantizar el acceso igualitario a la información y el conocimiento, y promover la innovación y el emprendimiento.
"""

In [23]:
ods_10 = """
La desigualdad de ingresos está en aumento - el 10 por ciento más rico de la población se queda hasta con el 40 por ciento del ingreso mundial total, mientras que el 10 por ciento más pobre obtiene solo entre el 2 y el 7 por ciento del ingreso total. En los países en desarrollo, la desigualdad ha aumentado un 11 por ciento, si se considera el aumento de la población.

La desigualdad de ingresos ha aumentado en casi todas partes en las últimas décadas, pero a diferentes velocidades. La más baja es en Europa y la más alta es en el Medio Oriente.

Para frenar este aumento de las disparidades, es necesario adoptar políticas sólidas que empoderen a las personas de bajos ingresos y promuevan la inclusión económica de todos y todas, independientemente de su género, raza o etnia.

La desigualad de ingresos es un problema mundial que requiere soluciones globales. Estas incluyen mejorar la regulación y el control de los mercados y las instituciones financieras y fomentar la asistencia para el desarrollo y la inversión extranjera directa para las regiones que más lo necesiten. Otro factor clave para salvar esta distancia es facilitar la migración y la movilidad segura de las personas.
"""

In [24]:
ods_11 = """
Más de la mitad de la población mundial vive hoy en zonas urbanas. En 2050, esa cifra habrá aumentado a 6.500 millones de personas, dos tercios de la humanidad. No es posible lograr un desarrollo sostenible sin transformar radicalmente la forma en que construimos y administramos los espacios urbanos.

El rápido crecimiento de las urbes en el mundo en desarrollo -como resultado de la creciente población y del incremento en la migración- ha provocado un incremento explosivo de las mega urbes, especialmente en el mundo desarrollado, y los barrios marginales se están convirtiendo en una característica más significativa de la vida urbana.  

Mejorar la seguridad y la sostenibilidad de las ciudades implica garantizar el acceso a viviendas seguras y asequibles y el mejoramiento de los asentamientos marginales. También incluye realizar inversiones en transporte público, crear áreas públicas verdes y mejorar la planificación y gestión urbana de manera que sea participativa e inclusiva.
"""

In [25]:
ods_12 = """
Para lograr crecimiento económico y desarrollo sostenible, es urgente reducir la huella ecológica mediante un cambio en los métodos de producción y consumo de bienes y recursos. La agricultura es el principal consumidor de agua en el mundo y el riego representa hoy casi el 70% de toda el agua dulce disponible para el consumo humano.

La gestión eficiente de los recursos naturales compartidos y la forma en que se eliminan los desechos tóxicos y los contaminantes son vitales para lograr este objetivo. También es importante instar a las industrias, los negocios y los consumidores a reciclar y reducir los desechos, como asimismo apoyar a los países en desarrollo a avanzar hacia patrones sostenibles de consumo para 2030.

El consumo de una gran proporción de la población mundial sigue siendo insuficiente para satisfacer incluso sus necesidades básicas. En este contexto, es importante reducir a la mitad el desperdicio per cápita de alimentos en el mundo a nivel de comercio minorista y consumidores para crear cadenas de producción y suministro más eficientes. Esto puede aportar a la seguridad alimentaria y llevarnos hacia una economía que utilice los recursos de manera más eficiente.
"""

In [26]:
ods_13 = """
No hay país en el mundo que no haya experimentado los dramáticos efectos del cambio climático. Las emisiones de gases de efecto invernadero continúan aumentando y hoy son un 50% superior al nivel de 1990. Además, el calentamiento global está provocando cambios permanentes en el sistema climático, cuyas consecuencias pueden ser irreversibles si no se toman medidas urgentes ahora.

Las pérdidas anuales promedio causadas solo por catástrofes relacionadas al clima alcanzan los cientos de miles de millones de dólares, sin mencionar el impacto humano de las catástrofes geofísicas, el 91 por ciento de las cuales son relacionadas al clima, y que entre 1998 y 2017 tomaron la vida de 1,3 millones de personas, y dejaron a 4.400 millones heridas. El objetivo busca movilizar US$ 100.000 millones anualmente hasta 2020, con el fin de abordar las necesidades de los países en desarrollo en cuanto a adaptación al cambio climático e inversión en el desarrollo bajo en carbono.

Apoyar a las regiones más vulnerables contriubuirá directamente no solo al Objetivo 13 sino tamién a otros Objetivos de Desarrollo Sostenible. Estas acciones deben ir de la mano con los esfuerzos destinados a integrar las medidas de reducción del riesgo de desastres en las políticas y estrategias nacionales. Con voluntad política y un amplio abanico de medidas tecnológicas, aún es posible limitar el aumento de la temperatura media global a dos grados Celsius por encima de los niveles pre-industriales, apuntando a 1,5°C. Para lograrlo, se requieren acciones colectivas urgentes.
"""

In [27]:
ods_14 = """
Los océanos del mundo, su temperatura, composición química, corrientes y vida son el motor de los sistemas globales que hacen que la Tierra sea un lugar habitable para los seres humanos. La forma en que gestionamos este recurso vital es fundamental para la humanidad y para contrarrestar los efectos del cambio climático.

Los medios de vida de más de 3.000 millones de personas dependen de la biodiversidad marina y costera. Sin embargo, el 30% de las poblaciones de peces del mundo está sobreexplotado, alcanzando un nivel muy por debajo del necesario para producir un rendimiento sostenible.

Los océanos también absorben alrededor del 30% del dióxido de carbón generado por las actividades humanas y se ha registrado un 26% de aumento en la acidificación de los mares desde el inicio de la revolución industrial. La contaminación marina, que proviene en su mayor parte de fuentes terrestres, ha llegado a niveles alarmantes: por cada kilómetro cuadrado de océano hay un promedio de 13.000 trozos de desechos plásticos.

Los Objetivos de Desarrollo Sostenible generan un marco para ordenar y proteger de manera sostenible los ecosistemas marinos y costeros de la contaminación terrestre, así como para abordar los impactos de la acidificación de los océanos. Mejorar la conservación y el uso sostenible de los recursos oceánicos a través del derecho internacional también ayudará a mitigar algunos de los retos que enfrentan los océanos.
"""

In [28]:
ods_15 = """
La vida humana depende de la tierra tanto como del océano para su sustento y subsistencia. La flora provee el 80% de la alimentación humana y la agricultura representa un recurso económico y un medio de desarrollo importante. A su vez, los bosques cubren el 30% de la superficie terrestre, proveen hábitats cruciales a millones de especies y son fuente importante de aire limpio y agua. Además, son fundamentales para combatir el cambio climático.

Cada año se pierden 13 millones de hectáreas de bosques, mientras que la degradación persistente de las tierras secas ha llevado a la desertificación de 3.600 millones de hectáreas, afectando desproporcionadamente a las comunidades pobres.

Mientras que el 15% de la tierra está protegida, la biodiversidad todavía está en riesgo. Cerca de 7.000 especies de animales y plantas han sido objeto del comercio ilegal. El tráfico de vida silvestre no solo erosiona la biodiversidad, sino que crea inseguridad, alimenta el conflicto y alimenta la corrupción.

Se deben tomar medidas urgentes para reducir la pérdida de hábitats naturales y biodiversidad que forman parte de nuestro patrimonio común y apoyar la seguridad alimentaria y del agua a nivel mundial, la mitigación y adaptación al cambio climático, y la paz y la seguridad.
"""

In [29]:
ods_16 = """
Sin paz, estabilidad, derechos humanos y gobernabilidad efectiva basada en el Estado de derecho, no es posible alcanzar el desarrollo sostenible. Vivimos en un mundo cada vez más dividido. Algunas regiones gozan de niveles permanentes de paz, seguridad y prosperidad, mientras que otras caen en ciclos aparentemente eternos de conflicto y violencia. De ninguna manera se trata de algo inevitable y debe ser abordado.

Los altos niveles de violencia armada e inseguridad tienen consecuencias destructivas para el desarrollo de un país, afectan el crecimiento económico y resultan a menudo en agravios arraigados que pueden extenderse por generaciones. La violencia sexual, los delitos, la explotación y la tortura también son fenómenos generalizados donde existen conflictos o no hay Estado de derecho y los países deben tomar medidas para proteger a los sectores que corren más riesgos.

Los Objetivos de Desarrollo Sostenible buscan reducir sustancialmente todas las formas de violencia y trabajan con los gobiernos y las comunidades para encontrar soluciones duraderas a los conflictos e inseguridad. El fortalecimiento del Estado de derecho y la promoción de los derechos humanos es fundamental en este proceso, así como la reducción del flujo de armas ilícitas y la consolidación de la participación de los países en desarrollo en las instituciones de gobernabilidad mundial.
"""

In [30]:
ods_17 = """
Los Objetivos de Desarrollo Sostenible solo se pueden lograr con el compromiso decidido a favor de alianzas mundiales y cooperación. La Asistencia Oficial para el Desarrollo se mantuvo estable pero por debajo del objetivo, a US$147.000 millones en 2017, mientras que las crisis humanitarias provocadas por conflictos o desastres naturales continúan demandando más recursos y ayuda financiera. Muchos países también requieren de esta asistencia para estimular el crecimiento y el intercambio comercial.

Hoy el mundo está más interconectado que nunca. Mejorar el acceso a la tecnología y los conocimientos es una forma importante de intercambiar ideas y propiciar la innovación. Para lograr el crecimiento y desarrollo sostenibles, es vital que se coordinen las políticas para ayudar a los países en desarrollo a manejar su deuda y para promover inversiones para los menos desarrollados.

La finalidad de los objetivos es mejorar la cooperación Norte-Sur y Sur-Sur, apoyando los planes nacionales en el cumplimiento de todas las metas. Promover el comercio internacional y ayudar a los países en desarrollo para que aumenten sus exportaciones, forma parte del desafío de lograr un sistema de comercio universal equitativo y basado en reglas que sea justo, abierto y beneficie a todos.
"""

In [31]:
def text_similarity(text):
    return [round(nlp(clean_tweets(text)).similarity(words),2) for words in words_list_nlp]

In [32]:
df_index = ['ods_' + str(i) for i in range(1,18)] + ['noticia', 'deportes', 'concierto']
df_index

['ods_1',
 'ods_2',
 'ods_3',
 'ods_4',
 'ods_5',
 'ods_6',
 'ods_7',
 'ods_8',
 'ods_9',
 'ods_10',
 'ods_11',
 'ods_12',
 'ods_13',
 'ods_14',
 'ods_15',
 'ods_16',
 'ods_17',
 'noticia',
 'deportes',
 'concierto']

In [33]:
df = pd.DataFrame([ods_1, ods_2, ods_3, ods_4, ods_5, ods_6, ods_7, ods_8, ods_9,
                   ods_10, ods_11, ods_12, ods_13, ods_14, ods_15, ods_16, ods_17,
                   noticia, deportes, concierto], index=df_index, columns=['text'])
df

,text
ods_1,erradicar pobreza formar principal desafío enf...
ods_2,rápido crecimiento económico aumentar producti...
ods_3,\nHemos logrado grandes avances en la lucha co...
ods_4,\nDesde 2000 se ha registrado un enorme progre...
ods_5,\nPoner fin a todas las formas de discriminaci...
ods_6,\nLa escasez de agua afecta a más del 40 por c...
ods_7,"\nEntre 2000 y 2016, la cantidad de personas c..."
ods_8,"\nDurante los últimos 25 años, la cantidad de ..."
ods_9,\nLa inversión en infraestructura y la innovac...
ods_10,\nLa desigualdad de ingresos está en aumento -...


In [34]:
classification = df['text'].apply(text_similarity)
classification

ods_1        [0.77, 0.78, 0.8, 0.74, 0.78, 0.57, 0.59, 0.83...
ods_2        [0.79, 0.82, 0.83, 0.74, 0.77, 0.62, 0.64, 0.8...
ods_3        [0.77, 0.78, 0.85, 0.75, 0.81, 0.56, 0.6, 0.85...
ods_4        [0.73, 0.75, 0.79, 0.79, 0.78, 0.56, 0.59, 0.8...
ods_5        [0.72, 0.71, 0.78, 0.74, 0.83, 0.49, 0.52, 0.8...
ods_6        [0.73, 0.8, 0.82, 0.72, 0.72, 0.69, 0.66, 0.82...
ods_7        [0.7, 0.79, 0.8, 0.7, 0.68, 0.68, 0.76, 0.81, ...
ods_8        [0.78, 0.75, 0.78, 0.73, 0.8, 0.56, 0.62, 0.88...
ods_9        [0.78, 0.78, 0.81, 0.82, 0.79, 0.63, 0.7, 0.89...
ods_10       [0.72, 0.73, 0.74, 0.7, 0.76, 0.58, 0.58, 0.83...
ods_11       [0.73, 0.76, 0.8, 0.75, 0.74, 0.63, 0.65, 0.83...
ods_12       [0.73, 0.82, 0.83, 0.73, 0.72, 0.68, 0.71, 0.8...
ods_13       [0.73, 0.73, 0.76, 0.69, 0.73, 0.61, 0.61, 0.8...
ods_14       [0.7, 0.77, 0.81, 0.67, 0.69, 0.71, 0.68, 0.79...
ods_15       [0.74, 0.82, 0.84, 0.66, 0.71, 0.71, 0.65, 0.8...
ods_16       [0.78, 0.74, 0.81, 0.74, 0.83, 0.57, 0.58,

In [35]:
df[df_index[:17]] = pd.DataFrame(classification.tolist(), index=df.index)
df.drop('text', axis=1)

,ods_1,ods_2,ods_3,ods_4,ods_5,ods_6,ods_7,ods_8,ods_9,ods_10,ods_11,ods_12,ods_13,ods_14,ods_15,ods_16,ods_17
ods_1,0.77,0.78,0.80,0.74,0.78,0.57,0.59,0.83,0.71,0.83,0.68,0.74,0.65,0.57,0.69,0.77,0.56
ods_2,0.79,0.82,0.83,0.74,0.77,0.62,0.64,0.85,0.76,0.82,0.74,0.79,0.69,0.63,0.75,0.76,0.61
ods_3,0.77,0.78,0.85,0.75,0.81,0.56,0.60,0.85,0.73,0.86,0.68,0.74,0.64,0.56,0.70,0.79,0.57
ods_4,0.73,0.75,0.79,0.79,0.78,0.56,0.59,0.83,0.72,0.80,0.68,0.73,0.63,0.55,0.69,0.74,0.57
ods_5,0.72,0.71,0.78,0.74,0.83,0.49,0.52,0.83,0.66,0.80,0.61,0.67,0.55,0.50,0.63,0.80,0.56
ods_6,0.73,0.80,0.82,0.72,0.72,0.69,0.66,0.82,0.74,0.77,0.71,0.81,0.73,0.61,0.72,0.71,0.50
ods_7,0.70,0.79,0.80,0.70,0.68,0.68,0.76,0.81,0.78,0.75,0.73,0.82,0.75,0.60,0.71,0.65,0.52
ods_8,0.78,0.75,0.78,0.73,0.80,0.56,0.62,0.88,0.76,0.85,0.69,0.78,0.65,0.53,0.66,0.76,0.62
ods_9,0.78,0.78,0.81,0.82,0.79,0.63,0.70,0.89,0.86,0.84,0.77,0.85,0.71,0.58,0.73,0.77,0.65
ods_10,0.72,0.73,0.74,0.70,0.76,0.58,0.58,0.83,0.71,0.80,0.70,0.75,0.65,0.54,0.66,0.70,0.57


In [36]:
with open('../doc/table.md', 'w') as f:
    f.write(df.drop('text', axis=1).to_markdown())